# **HANDWRITTEN DETECTION WITH PRE-TRAINED MODEL OF VISIONENCODERDECODERMODEL --- FINE TUNING MODEL**

<h2 align="center"><strong>IMAGE</strong></h2>
<img src="https://w7.pngwing.com/pngs/558/305/png-transparent-handwriting-digital-stamp-line-art-write-a-letter-miscellaneous-angle-text-thumbnail.png" width="300">

<img src="https://miro.medium.com/v2/resize:fit:1400/1*X1EzTTEMv14ETR-uJQzLcQ.png" width="600">


In [ ]:
!pip install transformers datasets torch==2.2.2 torchvision==0.17.2 pillow accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 105.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 42.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.wh

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('csv', data_files={'train': '/content/trainn.csv', 'test': '/content/test.csv'})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import TrOCRProcessor, AutoTokenizer
from PIL import Image

# Load a feature extractor and tokenizer
feature_extractor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Preprocess function
def preprocess_data(examples):
    images = [Image.open(image_path).convert("RGB") for image_path in examples['Nama File']]
    examples['pixel_values'] = feature_extractor(images, return_tensors="pt").pixel_values
    examples['labels'] = tokenizer(examples['Label'], padding="max_length", truncation=True, return_tensors="pt").input_ids
    return examples

# Apply the preprocessing
dataset = dataset.map(preprocess_data, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

In [ ]:
from transformers import VisionEncoderDecoderModel

model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')

config.json:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/FineTuned_VisionModel",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=4,
    logging_dir="/content/logs",
    logging_steps=10,
    eval_strategy="steps",
    save_steps=500,
    eval_steps=500,
    save_total_limit=2,
)

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    data_collator=data_collator,
    tokenizer=feature_extractor,
)

In [ ]:
result = trainer.train()

Step,Training Loss,Validation Loss


In [ ]:
import pandas as pd

# Evaluate the model
metrics = trainer.evaluate()

# Access the training logs to retrieve training loss
training_logs = trainer.state.log_history

# Extract the final training loss
final_train_loss = None
for log in reversed(training_logs):
    if "loss" in log:
        final_train_loss = log["loss"]
        break

# Add training loss to the evaluation metrics
if final_train_loss is not None:
    metrics["train_loss"] = final_train_loss

# Convert the metrics dictionary to a DataFrame
metrics_df = pd.DataFrame([metrics])

# Display the DataFrame
metrics_df

,eval_loss,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch,train_loss
0,0.044935,4.5708,2.625,0.656,4.0,0.0225


In [ ]:
import torch
torch.save(model.state_dict(), 'HTRVisionModel.pth')